### Imports and load the data

In [86]:
from showupforhealth.utils import perform_train_test_split, scale_df
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', 40)

from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV

# DL model with UNBALANCED data

### Load and shuffle the data

In [87]:
data = pd.read_csv('/Users/alessio/code/janduplessis883/data-showup/data/output-data/full_train_data.csv')
data = data.sample(frac = 1)
data

,Appointment_status,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
680127,1,17.2,0.0,63,0,0.14,0,1,0,0,0,0,0,21429.0,0.324254,0.128732,2.0,0,26.0,-0.748511,-6.631227e-01,-1.000000e+00,-1.836970e-16,-0.866025,-5.000000e-01,0.000000,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
307354,1,10.8,0.0,33,2,0.08,1,1,0,0,0,1,1,10748.0,0.525225,1.523371,18.0,0,31.0,-0.822984,5.680647e-01,1.224647e-16,-1.000000e+00,-0.500000,8.660254e-01,-0.433884,-0.900969,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
321149,1,19.1,0.0,59,0,0.28,1,0,0,0,0,0,0,8812.0,0.433298,0.904123,0.0,0,22.0,-0.568065,-8.229839e-01,-7.071068e-01,-7.071068e-01,-0.866025,-5.000000e-01,0.433884,-0.900969,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
169050,1,27.0,0.0,37,2,0.06,0,0,0,0,0,0,0,17464.0,0.668788,1.220002,6.0,0,3.0,-0.239316,-9.709418e-01,-7.071068e-01,-7.071068e-01,-0.500000,-8.660254e-01,0.974928,-0.222521,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
236994,1,5.4,0.0,73,0,0.06,0,0,0,0,1,0,0,17632.0,0.717432,0.584015,0.0,0,6.0,0.885456,4.647232e-01,2.588190e-01,-9.659258e-01,1.000000,6.123234e-17,-0.433884,-0.900969,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215600,1,5.7,0.0,34,2,0.11,0,1,0,0,0,0,0,13544.0,1.115830,0.289521,0.0,0,4.0,0.935016,3.546049e-01,5.000000e-01,-8.660254e-01,1.000000,6.123234e-17,0.781831,0.623490,15.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
407999,1,19.7,0.1,74,0,0.31,1,0,0,0,0,0,0,17632.0,0.751509,0.892772,7.0,0,22.0,-0.568065,-8.229839e-01,7.071068e-01,-7.071068e-01,-0.500000,-8.660254e-01,0.000000,1.000000,22.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
214599,1,8.4,0.0,40,0,0.19,0,0,0,0,0,0,0,4981.0,0.843333,0.075088,0.0,1,10.0,0.822984,5.680647e-01,-9.659258e-01,-2.588190e-01,0.866025,5.000000e-01,-0.433884,-0.900969,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
179232,1,16.8,0.1,31,0,0.03,1,0,0,0,0,0,0,13510.0,0.243210,0.675955,1.0,0,2.0,-1.000000,-1.836970e-16,2.588190e-01,-9.659258e-01,-0.866025,5.000000e-01,-0.974928,-0.222521,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [88]:
filt_sex_1 = (data['Sex'] != 1)
data = data[filt_sex_1]

In [89]:
filt_sex_2 = (data['Sex'] != 3)
data = data[filt_sex_2]
data['Sex'].value_counts()

0    571356
2    330086
Name: Sex, dtype: int64

In [90]:
data_dna = data[data['Appointment_status']==0][:10000]
data_dna.shape

(10000, 37)

In [91]:
data_no_dna = data[data['Appointment_status']==1][:10000]
data_no_dna.shape

(10000, 37)

In [92]:
data_balanced = pd.concat([data_dna, data_no_dna]).sample(frac = 1)
data_balanced['Appointment_status'].value_counts()

0    10000
1    10000
Name: Appointment_status, dtype: int64

In [93]:
data_balanced

,Appointment_status,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
12543,0,8.8,1.8,23,2,0.00,0,0,0,0,0,0,0,4330.0,0.109378,0.441702,15.0,1,1.0,6.432491e-16,1.000000,7.071068e-01,-7.071068e-01,-2.449294e-16,1.000000e+00,-0.433884,-0.900969,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
371322,1,9.7,0.0,67,0,0.28,0,0,1,0,1,1,0,23203.0,0.652980,0.682081,12.0,0,15.0,9.927089e-01,0.120537,7.071068e-01,-7.071068e-01,1.000000e+00,6.123234e-17,0.781831,0.623490,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
826422,1,-0.7,0.0,77,2,0.17,0,0,1,0,1,1,0,16808.0,0.288280,0.133181,5.0,0,2.0,8.854560e-01,0.464723,5.000000e-01,-8.660254e-01,8.660254e-01,5.000000e-01,0.781831,0.623490,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3514,0,20.1,0.0,68,2,0.14,0,0,1,1,1,0,0,16966.0,1.141740,1.590005,1.0,0,27.0,-8.229839e-01,-0.568065,-8.660254e-01,-5.000000e-01,-8.660254e-01,-5.000000e-01,0.433884,-0.900969,6.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
761772,1,5.7,0.0,31,2,0.03,0,0,0,0,0,0,0,3564.0,0.756166,0.641315,0.0,0,47.0,5.680647e-01,0.822984,-9.659258e-01,-2.588190e-01,5.000000e-01,8.660254e-01,0.000000,1.000000,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665271,1,15.1,0.0,79,0,0.00,0,0,0,0,0,0,0,24280.0,0.335082,0.429151,0.0,1,9.0,9.350162e-01,-0.354605,-1.000000e+00,-1.836970e-16,8.660254e-01,-5.000000e-01,0.433884,-0.900969,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
695756,1,7.2,0.0,61,2,0.11,0,0,0,0,1,0,0,11753.0,0.584078,0.725626,4.0,1,16.0,-5.680647e-01,0.822984,-7.071068e-01,-7.071068e-01,-5.000000e-01,8.660254e-01,0.000000,1.000000,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
230339,1,3.5,0.0,88,2,0.33,0,0,0,0,1,0,0,13544.0,1.072560,0.226446,3.0,0,34.0,3.546049e-01,0.935016,-7.071068e-01,-7.071068e-01,5.000000e-01,8.660254e-01,0.000000,1.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
754169,0,11.0,0.0,32,2,0.03,1,0,0,0,0,0,1,3890.0,0.376141,4.171371,9.0,0,15.0,-2.393157e-01,0.970942,5.000000e-01,-8.660254e-01,-2.449294e-16,1.000000e+00,0.974928,-0.222521,8.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Split in X and y

In [94]:
X = data_balanced.drop(columns=['Appointment_status'])
y = data_balanced['Appointment_status']

### Baseline accuracy

In [95]:
print(f'The baseline accuracy if we assume that all patients will show-up is {y.value_counts()[1] / X.shape[0]}')

The baseline accuracy if we assume that all patients will show-up is 0.5


### Train and Test split

In [96]:
X_train, X_test, y_train, y_test = perform_train_test_split(X,y)

✅ OUTPUT: X_train, X_test, y_train, y_test
Train Set:  X_train, y_train - (16000, 36), (16000,)
 Test Set:  X_test, y_test - - (4000, 36), (4000,)


### Scale X_train and X_test

In [97]:
def mm_scaler(X_train, X_test):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

X_train_scaled, X_test_scaled = mm_scaler(X_train, X_test)

In [98]:
X_train_ready = pd.DataFrame(X_train_scaled,columns=X.columns)
X_test_ready = pd.DataFrame(X_test_scaled,columns=X.columns)

# Models

### SVM - Support Vector Machine

In [15]:
from sklearn import svm
from sklearn.metrics import f1_score
svm = svm.SVC(kernel='poly')
grid = {'C':[2000,3000,4000,5000],
    'gamma': [0.1, 0.01, 0.001],
        'degree':[1,2,3]}

search_svm = GridSearchCV(svm, grid, scoring = 'precision', cv = 5, n_jobs=-1)

search_svm.fit(X_train_ready, y_train)
search_svm_prediction = search_svm.predict(X_test_ready) 

print(search_svm.best_params_)
print(classification_report(y_test, search_svm_prediction)) 
print(confusion_matrix(y_test, search_svm_prediction))
print(f1_score(y_test, search_svm_prediction, labels=np.unique(search_svm_prediction)))

{'C': 4000, 'degree': 1, 'gamma': 0.1}
              precision    recall  f1-score   support

           0       0.74      0.73      0.74       203
           1       0.73      0.74      0.73       197

    accuracy                           0.73       400
   macro avg       0.74      0.74      0.73       400
weighted avg       0.74      0.73      0.74       400

[[148  55]
 [ 51 146]]
0.7336683417085428


In [67]:
svm_best = svm.SVC(kernel='poly', C= 3000, degree= 2, gamma=0.1)
svm_best_cv = cross_val_score(svm_best,X_train_scaled,y_train,cv=5,scoring='precision',n_jobs=-1).mean()
svm_best_cv

### SGDClassifier

In [88]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter=100000)
grid = {'loss': ('hinge', 'log_loss', 'modified_huber', 'perceptron', 'huber', 'epsilon_insensitive'), 
         'penalty': ('l2', 'l1', 'elasticnet', None),
         'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1]}

search_sgd = GridSearchCV(sgd, grid, scoring = 'precision', cv = 5, n_jobs=-1)

search_sgd.fit(X_train_ready, y_train)
search_sgd_prediction = search_sgd.predict(X_test_ready) 

print(search_sgd.best_params_) 
print(classification_report(y_test, search_sgd_prediction)) 
print(confusion_matrix(y_test, search_sgd_prediction))

/Users/alessio/.pyenv/versions/3.10.6/envs/showupforhealth/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alessio/.pyenv/versions/3.10.6/envs/showupforhealth/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alessio/.pyenv/versions/3.10.6/envs/showupforhealth/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

{'alpha': 1e-05, 'loss': 'modified_huber', 'penalty': 'elasticnet'}
              precision    recall  f1-score   support

           0       0.73      0.77      0.75      5960
           1       0.76      0.72      0.74      6040

    accuracy                           0.75     12000
   macro avg       0.75      0.75      0.75     12000
weighted avg       0.75      0.75      0.75     12000

[[4594 1366]
 [1672 4368]]


### Decision Tree Classifier

In [87]:
%%time
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()

grid = {'criterion':['gini', 'entropy'],
         'max_depth': [1,2,3],
          'min_samples_split':[1,2,3],
          'min_samples_leaf': [1,2,3,4]}

tree_search = GridSearchCV(tree, grid, scoring ='precision', cv = 5, n_jobs=-1)

tree_search.fit(X_train_ready, y_train)
tree_search_prediction = tree_search.predict(X_test_ready) 

print(tree_search.best_params_) 
print(classification_report(y_test, tree_search_prediction)) 
print(confusion_matrix(y_test, tree_search_prediction))

{'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 1}
              precision    recall  f1-score   support

           0       0.64      1.00      0.78      5960
           1       1.00      0.45      0.62      6040

    accuracy                           0.72     12000
   macro avg       0.82      0.73      0.70     12000
weighted avg       0.82      0.72      0.70     12000

[[5960    0]
 [3303 2737]]
CPU times: user 885 ms, sys: 168 ms, total: 1.05 s
Wall time: 6.04 s


### XGBoost 1

In [73]:
import xgboost as xgb
estimator = xgb.XGBClassifier(objective= 'binary:logistic', nthread=4, seed=42)

grid = {'max_depth': [2,3,4,5,6,7,8,9,10],
        'n_estimators': [60,80,100,120,140,160,180,200,220],
        'learning_rate': [2, 1, 0.1, 0.01, 0.05]}

xgb_search = GridSearchCV(
    estimator=estimator,
    param_grid=grid,
    scoring = 'precision',
    n_jobs = -1,
    cv = 5,
    verbose=True)

xgb_search.fit(X_train_scaled, y_train)

xgb_search_prediction = xgb_search.predict(X_test_scaled) 
xgb_search_best = xgb_search.best_estimator_

print(xgb_search.best_params_) 
print(classification_report(y_test, xgb_search_prediction)) 
print(confusion_matrix(y_test, xgb_search_prediction))

Fitting 5 folds for each of 405 candidates, totalling 2025 fits
{'learning_rate': 1, 'max_depth': 4, 'n_estimators': 200}
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      2029
           1       0.96      0.99      0.97      1971

    accuracy                           0.97      4000
   macro avg       0.98      0.98      0.97      4000
weighted avg       0.98      0.97      0.98      4000

[[1956   73]
 [  27 1944]]


In [57]:

xgb_search_cv = cross_val_score(xgb_search_best, X, y, cv=5, scoring='precision').mean()
xgb_search_cv

0.8710222214674694

### XGBoost 2

In [ ]:
# 'min_child_weight': [1, 5, 10] 'subsample': [0.6, 0.8, 1.0]'colsample_bytree': [0.6, 0.8, 1.0]

In [101]:
%%time
import xgboost as xgb
xgb2 = xgb.XGBClassifier(objective= 'binary:logistic', random_state=42, learning_rate=0.05)

grid = {'gamma': [0.05, 0.1, 0.5],
        'max_depth': [4, 5, 6, 7, 8],
        'n_estimators': [140, 160, 180]}

xgb2_search = GridSearchCV(estimator=xgb2, param_grid=grid, scoring='accuracy', n_jobs=-1, cv = 5, verbose=True)

xgb2_search.fit(X_train_ready, y_train)

xgb2_search_best = xgb2_search.best_estimator_

xgb2_search_prediction = xgb2_search.predict(X_test_ready) 

print(xgb2_search.best_params_) 
print(classification_report(y_test, xgb2_search_prediction)) 
print(confusion_matrix(y_test, xgb2_search_prediction))
print(f1_score(y_test, xgb2_search_prediction, labels=np.unique(xgb2_search_prediction)))

Fitting 5 folds for each of 45 candidates, totalling 225 fits
{'gamma': 0.05, 'max_depth': 7, 'n_estimators': 160}
              precision    recall  f1-score   support

           0       0.79      0.90      0.84      1971
           1       0.89      0.77      0.83      2029

    accuracy                           0.83      4000
   macro avg       0.84      0.84      0.83      4000
weighted avg       0.84      0.83      0.83      4000

[[1773  198]
 [ 464 1565]]
0.82542194092827
CPU times: user 19.4 s, sys: 1.14 s, total: 20.6 s
Wall time: 5min 10s


### Logistic regression

In [28]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=1000)
score_log_reg = cross_val_score(log_reg, X, y, cv=5, scoring='precision').mean()
score_log_reg

/Users/alessio/.pyenv/versions/3.10.6/envs/showupforhealth/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/alessio/.pyenv/versions/3.10.6/envs/showupforhealth/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

0.8301249573013255

### GaussianNB

In [29]:
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
score_gaussian = cross_val_score(gaussian, X, y, cv=5, scoring='precision').mean()
score_gaussian

0.8128434150658024